In [0]:
import tensorflow as tf
import numpy as np

In [0]:
(cifar_x_train, cifar_y_train),(cifar_x_test, cifar_y_test) = tf.keras.datasets.cifar10.load_data()

In [0]:
def conv_batch_norm(x, n_out, phase_train):

	#Step 1 : Grab the logits before they go through the non-linearity
	#Step 2: You have to normalize them by subtracting mean and dividing by standard deviation
	#Step 3: to restore representational power of the vectors, affine transform the vectors with : gamma*x+beta
  beta_init = tf.constant_initializer(value=0.0,dtype=tf.float32)
  gamma_init = tf.constant_initializer(value=1.0,dtype=tf.float32)
  
  beta = tf.get_variable("beta", [n_out],initializer=beta_init)
  gamma = tf.get_variable("gamma", [n_out],initializer=gamma_init)
  
  batch_mean, batch_var = tf.nn.moments(x, [0,1,2],name='moments')
  ema = tf.train.ExponentialMovingAverage(decay=0.9)
  ema_apply_op = ema.apply([batch_mean, batch_var])
  ema_mean, ema_var = ema.average(batch_mean),ema.average(batch_var)
  
  def mean_var_with_update():
    with tf.control_dependencies([ema_apply_op]):
      return tf.identity(batch_mean),tf.identity(batch_var)
    
  mean, var = tf.cond(phase_train,mean_var_with_update,lambda: (ema_mean, ema_var))
  normed = tf.nn.batch_norm_with_global_normalization(x,mean, var, beta, gamma, 1e-3, True)
  return normed

In [0]:
def layer_batch_norm(x, n_out, phase_train):
    
  beta_init = tf.constant_initializer(value=0.0,dtype=tf.float32)
  gamma_init = tf.constant_initializer(value=1.0,dtype=tf.float32)
  
  beta = tf.get_variable("beta", [n_out],initializer=beta_init)
  gamma = tf.get_variable("gamma", [n_out],initializer=gamma_init)
  batch_mean, batch_var = tf.nn.moments(x, [0],name='moments')
  
  ema = tf.train.ExponentialMovingAverage(decay=0.9)
  ema_apply_op = ema.apply([batch_mean, batch_var])
  ema_mean, ema_var = ema.average(batch_mean),ema.average(batch_var)
  
  def mean_var_with_update():
    with tf.control_dependencies([ema_apply_op]):
      return tf.identity(batch_mean),tf.identity(batch_var)
  
  mean, var = tf.cond(phase_train,mean_var_with_update,lambda: (ema_mean, ema_var))
  x_r = tf.reshape(x, [-1, 1, 1, n_out])
  normed = tf.nn.batch_norm_with_global_normalization(x_r,mean, var, beta, gamma, 1e-3, True)
  return tf.reshape(normed, [-1, n_out])


In [0]:
def layer(input, weight_shape, bias_shape, phase_train):
  
  weight_std = (2/weight_shape[0])**0.5
  weight_init = tf.random_normal_initializer(stddev = weight_std)
  bias_init = tf.constant_initializer(value=0)

  W = tf.get_variable("W", weight_shape, initializer=weight_init)
  b = tf.get_variable("b", bias_shape, initializer=bias_init)
    
  logits = tf.matmul(input,W)+b
  return tf.nn.relu(layer_batch_norm(logits, weight_shape[1], phase_train))

In [0]:
def conv2d (input, weight_shape, bias_shape, phase_train, visualize=False):
  
  inp = weight_shape[0] * weight_shape[1] * weight_shape[2]
  weight_init = tf.random_normal_initializer(stddev = (2.0/inp)**0.5)
    
  W = tf.get_variable("W", weight_shape, initializer = weight_init)
    
  bias_init = tf.constant_initializer(value=0)
  b = tf.get_variable("b", bias_shape, initializer=bias_init)
   
  conv_out = tf.nn.conv2d(input, W, strides=[1,1,1,1], padding='SAME')
  logits = tf.nn.bias_add(conv_out,b)
  return tf.nn.relu(conv_batch_norm(logits, weight_shape[3], phase_train))

In [0]:
def max_pool(input,k=2):
  return tf.nn.max_pool(input, ksize = [1,k,k,1], strides = [1,k,k,1], padding='SAME')

In [0]:
def inference(x, keep_prob, phase_train):
  x = tf.reshape(x,[-1,32,32,3])
  with tf.variable_scope("conv_1"):
    conv_1 = conv2d(x,[5,5,3,64], [64], phase_train)
    max_pool_1 = max_pool(conv_1)

  with tf.variable_scope("conv_2"):
    conv_2 = conv2d(max_pool_1,[5,5,64,64],[64], phase_train)
    max_pool_2 = max_pool(conv_2)
  
  with tf.variable_scope("fc_1"):
    dim = 1
    for d in max_pool_2.get_shape()[1:].as_list():
      dim *= d
    pool_2_flat = tf.reshape(max_pool_2,[-1, dim])
    fc1 = layer(pool_2_flat,[dim, 384],[384], phase_train)
        
    fc1_dropout = tf.nn.dropout(fc1, keep_prob)
    
  with tf.variable_scope("fc_2"):
    
    fc2 = layer(fc1_dropout,[384,192],[192], phase_train)
        
    fc2_dropout = tf.nn.dropout(fc2, keep_prob)
    
    with tf.variable_scope("output"):
        output = layer(fc_2dropout, [192,10], [10])
    
    return output

In [0]:
def loss(output,y):
  xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=output)
  loss = tf.reduce_mean(xentropy)
  return loss

# optimizer which will reduce the error and update gradients
def training(cost, global_step):
  tf.summary.scalar("cost",cost)

  optimizer = tf.train.AdamOptimizer(learning_rate)
  train_op = optimizer.minimize(cost,global_step=global_step)
  return train_op

#evaluation of the model
def evaluation(output,y):
  correct_pred = tf.equal(tf.argmax(output,1),tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
  return accuracy

In [0]:
#Parameters 
learning_rate = 0.01
training_epochs = 300
batch_size = 150
display_step =20

In [0]:
with tf.Graph().as_default():
  print(cifar_x_train[0].shape)
  x = tf.placeholder_with_default(False, shape=())
  x = tf.cast(x,tf.float32)

  #digits recognition placeholder
  y = tf.placeholder_with_default(False, shape=())
  y = tf.cast(y,tf.float32)

  output = inference(x,1,0.5)
  cost = loss(output,y)
  global_step = tf.Variable(0,name="global_step",trainable=False)

  train_op = training(cost,global_step)
  eval_op = evaluation(output,y)

  summary_op = tf.summary.merge_all()

  saver = tf.train.Saver()
  sess = tf.Session()
  summary_writer = tf.summary.FileWriter("feed_forward_logs/",graph_def = sess.graph_def)

  init_op = tf.initialize_all_variables()

  sess.run(init_op)

    #Training Cycle

  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)

    #Loop over all batches
    for i in range(total_batch):
      mbatch_x, mbatch_y = cifar_x_train.next_batch(batch_size), cifar_y_train.next_batch(batc_size)

      feed_dict = {x: mbatch_x, y:mbatch_y}
      sess.run(train_op, feed_dict= feed_dict)

      minibatch_cost = sess.run(cost, feed_dict= feed_dict)
      avg_cost += minibatch_cost/total_batch

  print ("Optimization Finished")

  test_feed_dict = {
        x: cifar_x_test,
        y: cifar_y_test
    }

  accuracy = sess.run(eval_op, feed_dict = test_feed_dict)

  print ("Testing accuracy: {}".format(accuracy))